# 🚀 **Pipeline PREMIUM MODEL : Recommandation Sémantique Avancée**
*Architecture Hybride : Deep Learning (SBERT) + Métadonnées Pondérées + KNN*

---

Ce notebook implémente un système de recommandation de **"nouvelle génération" (Content-Based)** qui dépasse les limites de la recherche par mots-clés classiques (TF-IDF). Contrairement aux approches lexicales, ce modèle utilise des réseaux de neurones pour comprendre le **sens profond** et le **contexte** des synopsis d'animes.

---

## 🧠 **1. Architecture du Modèle**

Le modèle construit une **"carte d'identité mathématique"** (vecteur dense) pour chaque anime en fusionnant trois sources d'information distinctes :

### **A. Le Cerveau (Sémantique - SBERT)** 🧠
* **Technologie :** `sentence-transformers/all-MiniLM-L6-v2` (Transformer).
* **Rôle :** Transforme le texte du synopsis en un vecteur dense de **384 dimensions**.
* **Avantage :** Capture la sémantique. Il comprend que *"Samouraï"* et *"Katana"* sont liés, ou que *"École"* et *"Lycée"* sont synonymes, même sans mot-clé commun.

### **B. Le Squelette (Genres - Pondérés)** 🦴
* **Technologie :** One-Hot Encoding.
* **Pondération :** **x0.6** *(Ajustable)*.
* **Rôle :** Assure la **cohérence de catégorie**.
* **Pourquoi ?** Empêche de recommander une *Romance* à un fan d'*Horreur* simplement parce que le mot "cœur" (organe vs amour) apparaît dans les deux résumés.

### **C. La "Vibe" (Statistiques - Pondérées)** 📊
* **Technologie :** Normalisation MinMax sur `Score`, `Membres`, `Épisodes`.
* **Pondération :** **x0.2** *(Ajustable)*.
* **Rôle :** Filtre par **qualité** et **popularité**.
* **Pourquoi ?** Permet de recommander des blockbusters si l'utilisateur cherche un blockbuster, ou des pépites de niche si l'utilisateur cherche une niche.

---

## ⚙️ **2. Logique de Recommandation (Inférence)**

L'algorithme de recherche ne se contente pas de trouver le voisin le plus proche. Il suit une **logique en cascade** pour maximiser la pertinence utilisateur :

1.  **Recherche Intelligente (Entrée) :**
    * 🎯 **Priorité 1 : Match Exact** sur le titre.
    * ⚠️ **Priorité 2 : Match Partiel + Tri par Popularité** (pour éviter de proposer un film obscur ou un OVA quand l'utilisateur cherche la série principale).

2.  **Calcul de Similarité (Cœur) :**
    * Utilisation de **KNN (K-Nearest Neighbors)**.
    * Métrique : **Cosine Similarity**.
    * Comparaison des vecteurs fusionnés *(Texte + Genre + Stats)*.

---

## 📂 **3. Sorties Attendues (Outputs)**

Le pipeline sauvegarde les artefacts dans un dossier horodaté (`runs/content_premium/YYYYMMDD_HHMMSS/`) pour assurer la reproductibilité :

| Fichier | Description |
| :--- | :--- |
| **`features_premium.npz`** | La matrice finale des embeddings (**Dense**). Contient toute l'intelligence du modèle. |
| **`knn_model_premium.joblib`** | Le modèle KNN entraîné, prêt à trouver les voisins. |
| **`genre_binarizer.joblib`** | L'encodeur des genres (pour traiter de nouveaux animes futurs). |
| **`numerical_scaler.joblib`** | Le normalisateur des statistiques (MinMax Scaler). |
| **`training_data.csv`** | Snapshot des données utilisées (Indispensable pour mapper `Index` ↔ `Titre`). |

In [2]:
# ============================================================
# Notebook : 04_train_content_semantic
# Objectif : Entraînement du modèle Content-Based PREMIUM
# Pipeline : SBERT (Deep Learning) + Genres + Numerical → KNN
# ============================================================

import os
import yaml
import joblib
import pandas as pd
import numpy as np
import ast

# Deep Learning pour le NLP
from sentence_transformers import SentenceTransformer

from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors
from scipy import sparse
from datetime import datetime

In [3]:
# ============================================================
# 1. Configuration & Chargement
# ============================================================

# Chemin à adapter selon ton environnement
config_path = "config/content_premium.yaml" 
# Si tu n'as pas de yaml premium, utilise celui du light pour l'instant
# ou crée-en un avec : model: { transformer: "all-MiniLM-L6-v2" }

print("Loading configuration...")
# Simulation de config pour le script si le fichier n'existe pas encore
config = {
    "model": {
        "transformer_name": "all-MiniLM-L6-v2", # Petit, rapide, puissant
        "knn": {"n_neighbors": 20}
    }
}

# Chargement Dataset
df = pd.read_csv("../data/processed/anime_master_clean.csv")
df = df.dropna(subset=["synopsis"])
df = df.reset_index(drop=True)
print(f"Dataset chargé : {df.shape}")

# Nettoyage des genres
def fix_list(x):
    if isinstance(x, str):
        try: return ast.literal_eval(x)
        except: return []
    return x

df["genres_list"] = df["genres_list"].apply(fix_list)
df["genres_list"] = df["genres_list"].apply(lambda lst: [g.strip() for g in lst])
df["genres_str"] = df["genres_list"].apply(lambda lst: ", ".join(lst))

Loading configuration...
Dataset chargé : (18266, 20)


In [4]:
# ============================================================
# 2. Run Directory
# ============================================================

run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = f"runs/content_premium/{run_id}"
artifacts_dir = os.path.join(run_dir, "artifacts")
os.makedirs(artifacts_dir, exist_ok=True)

In [5]:
# ============================================================
# 3. Feature Engineering : SBERT (Le cœur du Premium)
# ============================================================

print("⏳ Chargement du modèle SBERT (peut prendre quelques secondes la 1ère fois)...")
# Ce modèle transforme les phrases en vecteurs de 384 dimensions
model_name = config["model"]["transformer_name"]
sbert_model = SentenceTransformer(model_name)

print("🧠 Encodage des synopsis en cours (Deep Learning)...")
# Contrairement à TF-IDF, c'est un peu plus long, mais bien meilleur
synopsis_embeddings = sbert_model.encode(
    df["synopsis"].fillna("").tolist(), 
    batch_size=64, 
    show_progress_bar=True, 
    convert_to_numpy=True
)

print(f"Embeddings SBERT terminés : {synopsis_embeddings.shape}")

⏳ Chargement du modèle SBERT (peut prendre quelques secondes la 1ère fois)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Marvin\Documents\HEPL\MASI 2\Machine learning\anime-recommender-\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Marvin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🧠 Encodage des synopsis en cours (Deep Learning)...


Batches:   0%|          | 0/286 [00:00<?, ?it/s]

Embeddings SBERT terminés : (18266, 384)


In [14]:
# ============================================================
# 4. Feature Engineering : Metadata (Genres + Numérique)
# ============================================================

# --- A. GENRES ---
print("⚙️ Traitement des genres...")
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(df["genres_list"])

# Application du poids (Weighting)
try:
    w_genres = config["model"]["weights"]["genres"]
except:
    w_genres = 0.6 # Valeur par défaut si config absente
    
genre_matrix = genre_matrix * w_genres


# --- B. NUMÉRIQUE (La version robuste) ---
print("⚙️ Traitement des données numériques...")

# Liste idéale des colonnes qu'on veut
target_cols = ["score", "members", "episodes", "popularity"]

# FILTRE DE SÉCURITÉ : On ne garde que celles qui existent vraiment dans ton CSV
num_cols = [c for c in target_cols if c in df.columns]

print(f"📊 Colonnes numériques trouvées et utilisées : {num_cols}")

if len(num_cols) > 0:
    scaler = MinMaxScaler()
    num_scaled = scaler.fit_transform(df[num_cols].fillna(0))
    
    # Application du poids
    try:
        w_num = config["model"]["weights"]["numerical"]
    except:
        w_num = 0.2
        
    num_scaled = num_scaled * w_num
else:
    # Cas de secours si aucune colonne n'existe (peu probable)
    print("⚠️ Attention : Aucune colonne numérique trouvée.")
    num_scaled = np.zeros((len(df), 0))

⚙️ Traitement des genres...
⚙️ Traitement des données numériques...
📊 Colonnes numériques trouvées et utilisées : ['score', 'members', 'popularity']


In [15]:
# ============================================================
# 5. Fusion des Features
# ============================================================

# On concatène tout : [Sens du texte (384)] + [Genres (~40)] + [Stats (3)]
# Note: SBERT donne des arrays dense, pas sparse. On passe tout en numpy array standard.
features = np.hstack([synopsis_embeddings, genre_matrix, num_scaled])

# Normalisation finale pour que la distance Cosine fonctionne parfaitement
from sklearn.preprocessing import normalize
features = normalize(features, norm='l2', axis=1)

print(f"✅ Matrice finale de features : {features.shape}")

✅ Matrice finale de features : (18266, 408)


In [16]:
# ============================================================
# 6. Entraînement KNN
# ============================================================

print("🚀 Entraînement du KNN...")
knn = NearestNeighbors(
    n_neighbors=config["model"]["knn"]["n_neighbors"],
    metric="cosine",
    algorithm="brute"
)
knn.fit(features)

🚀 Entraînement du KNN...


,n_neighbors,20
,radius,1.0
,algorithm,'brute'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [17]:
# ============================================================
# 7. Sauvegarde des Artefacts
# ============================================================

print(f"💾 Sauvegarde dans {artifacts_dir}...")

# On sauvegarde la matrice features (on utilise numpy save compressé car c'est dense)
np.savez_compressed(f"{artifacts_dir}/features_premium.npz", features)

# Sauvegarde du modèle KNN
joblib.dump(knn, f"{artifacts_dir}/knn_model_premium.joblib")

# Sauvegarde des objets de pré-traitement pour l'inférence future
# Note : Pour SBERT, on ne sauvegarde pas le modèle, on rechargera "all-MiniLM-L6-v2"
joblib.dump(mlb, f"{artifacts_dir}/genre_binarizer.joblib")
joblib.dump(scaler, f"{artifacts_dir}/numerical_scaler.joblib")
df.to_csv(f"{artifacts_dir}/training_data.csv", index=False)

💾 Sauvegarde dans runs/content_premium/20251221_205355\artifacts...


In [62]:
# ============================================================
# 8. Test de Recommandation
# ============================================================

def find_best_match(title, df):
    # 1. Nettoyage de la requête utilisateur (minuscule + sans espaces autour)
    search_term = title.strip().lower()
    
    # --- ÉTAPE 1 : RECHERCHE EXACTE (Priorité absolue) ---
    # On cherche si un titre correspond parfaitement (en ignorant la casse majuscule/minuscule)
    exact_match = df[df["title"].str.lower() == search_term]
    
    if not exact_match.empty:
        # Si on a plusieurs matchs exacts (rare, mais possible), on prend le plus populaire
        # C'est souvent la série TV principale
        best_idx = exact_match.sort_values(by="members", ascending=False).index[0]
        print(f"🎯 Match Exact trouvé : {df.loc[best_idx, 'title']}")
        return best_idx

    # --- ÉTAPE 2 : RECHERCHE PARTIELLE (Plan B) ---
    # Si pas de match exact, on cherche "ce qui contient le mot"
    partial_match = df[df["title"].str.lower().str.contains(search_term, regex=False, na=False)]
    
    if partial_match.empty:
        return None

    # --- ÉTAPE 3 : TRI INTELLIGENT ---
    # Parmi les résultats partiels, on veut éviter "One Piece Movie 3" si la série existe.
    # On trie par nombre de membres (popularité) pour faire remonter la série principale.
    best_idx = partial_match.sort_values(by="members", ascending=False).index[0]
    
    print(f"⚠️ Pas de match exact. Meilleur résultat partiel trouvé : {df.loc[best_idx, 'title']}")
    return best_idx

def recommend_premium(title, k=30):
    idx = find_best_match(title, df)
    
    # Gestion d'erreur si l'anime n'existe pas du tout
    if idx is None:
        return f"Désolé, l'anime '{title}' est introuvable."
    
    # Le reste ne change pas...
    print(f"🔍 Recommandations basées sur : {df.loc[idx, 'title']}")
    
    # Inférence KNN
    # Attention: on met k+1 car le premier résultat est toujours l'anime lui-même
    distances, indices = knn.kneighbors([features[idx]], n_neighbors=k+1)
    
    results = []
    # On commence à [1:] pour sauter l'anime lui-même (distance 0)
    for i, dist in zip(indices[0][1:], distances[0][1:]):
        score_sim = 1 - dist # Convertir distance en similarité
        results.append({
            "title": df.loc[i, "title"],
            "similarity": round(score_sim, 4),
            "genres": df.loc[i, "genres_str"],
            "score": df.loc[i, "score"],
            "mal_id": df.loc[i, "mal_id"] # Utile si tu veux faire des liens plus tard
        })
    
    return pd.DataFrame(results)

In [66]:
# Test sur un anime psychologique complexe (là où TF-IDF échoue souvent)
print(recommend_premium("jujutsu kaisen"))


🎯 Match Exact trouvé : Jujutsu Kaisen
🔍 Recommandations basées sur : Jujutsu Kaisen
                                                title  similarity  \
0                                    Kimetsu no Yaiba      0.7307   
1                                          Onigamiden      0.6959   
2                              Jujutsu Kaisen 0 Movie      0.6885   
3                           Jujutsu Kaisen 2nd Season      0.6883   
4                                       Yumekui Merry      0.6509   
5                                      Ao no Exorcist      0.6467   
6                                            Amatsuki      0.6441   
7                  Kimetsu no Yaiba: Mugen Ressha-hen      0.6429   
8            Kimetsu no Yaiba: Katanakaji no Sato-hen      0.6390   
9                       Kimetsu no Yaiba: Yuukaku-hen      0.6388   
10                                    Yuu☆Yuu☆Hakusho      0.6291   
11             Ao no Exorcist: Shimane Illuminati-hen      0.6289   
12                 

# Comparaison des 2 modèles : 
## 1. Le "Top 1" est identique (et c'est rassurant)
Dans les deux cas, Kimetsu no Yaiba (Demon Slayer) est le #1.

Pourquoi ? C'est logique. Jujutsu Kaisen et Demon Slayer sont les deux piliers du "Shonen moderne de chasseurs de démons".

Conclusion : Tes deux modèles ont bien compris la base. C'est un bon signe de stabilité.

## 2. La défaillance du modèle Light (L'erreur "Arrietty") ❌
Regarde attentivement la liste du modèle Light. Il recommande "Karigurashi no Arrietty" (Arrietty, le petit monde des chapardeurs) avec un score de 0.666.

Le problème : Jujutsu Kaisen est un anime violent, sombre, avec des combats sanglants et des malédictions. Arrietty est un film du Studio Ghibli, doux, poétique, sur des petits êtres qui vivent sous le plancher. C'est une erreur de recommandation flagrante.

Pourquoi TF-IDF a fait ça ? Il a probablement trouvé des mots-clés communs dans le synopsis comme "humain", "vivre caché", "secret" ou "danger". Il a matché les mots, pas l'ambiance.

## 3. La finesse du modèle Premium (La cohérence Sémantique) ✅
Ton nouveau modèle (SBERT) ne recommande pas Arrietty. Il a compris que le sens du synopsis d'Arrietty n'avait rien à voir avec celui de JJK.

À la place, il fait remonter des titres plus pertinents thématiquement :

Sword Gai The Animation (0.614) : C'est un ajout très intéressant du modèle Premium. L'histoire parle d'un garçon qui fusionne avec une épée légendaire/maudite et combat des monstres.

Lien Sémantique : Dans JJK, Yuji fusionne avec un objet maudit (le doigt de Sukuna) et utilise des outils maudits. SBERT a capté ce lien "Fusion / Objet Maudit" que TF-IDF a raté.

Yumekui Merry (0.650) : Un héros qui peut voir des "rêves" (monstres) envahir le monde réel.

Lien Sémantique : Très proche du concept de JJK où les "Malédictions" (invisibles aux humains normaux) envahissent le quotidien.

## 4. La gestion de la Franchise (Suites et Films)
Regarde comment ils classent les autres contenus Jujutsu Kaisen :

Modèle Light : Met la Saison 2 en premier, puis le film JJK 0 plus bas.

Modèle Premium : Met le film JJK 0 (0.6885) juste devant la Saison 2 (0.6883).

Analyse : C'est subtil, mais le synopsis de JJK 0 (l'histoire de Yuta Okkotsu) est structurellement plus proche du début de JJK (l'histoire de Yuji Itadori) : un lycéen normal se retrouve avec une malédiction surpuissante et rejoint l'école d'exorcisme. SBERT a détecté cette similarité narrative structurelle très forte.
